# Using undersampling to reequilibrate the classes, and the reiterate a selection of stacking model on the rebalanced dataset.

**Strategy:**
- For each validation set / For each model:
    - Undersample the majority category
    - Train select the best hyperparameters for various models
    - Stack the models
    - Apply on the full test dataset

**To be tried**
- Tomek link (When classes overlap)
- NearMiss (When decision boundary matters)

## Library loading

In [17]:
# path managememnt
import os

# print path
print("Current working directory:", os.getcwd())
# change path
os.chdir("C:/Users/axel-/Documents/Coding/SISE_FraudAnalysis")
os.getcwd()

Current working directory: c:\Users\axel-\Documents\Coding


'C:\\Users\\axel-\\Documents\\Coding\\SISE_FraudAnalysis'

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import pickle\

## Data loading

In [21]:
with open('data/cleaned_data.pkl', 'rb') as f:
    df = pickle.load(f)

In [20]:
df.head()

,ZIBZIN,IDAvisAutorisationCheque,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNb_RB,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr1,DiffDateTr2,DiffDateTr3,CA3TRetMtt,CA3TR,Heure,JourSemaine
0,A013010004908126703060931,78643044,0,20.00,2017-02-01 07:32:14,1,0,0,0,551,0.000000,0.000000,0.000000,37.186668,52.076034,0,1,4.000000,4.0,4.0,20.00,0.00,27134,2
1,A013011306908024927155000,78643045,0,20.00,2017-02-01 07:43:37,1,0,0,0,551,0.000000,0.000000,0.000000,48.844716,52.076034,1,2,1.797685,4.0,4.0,28.61,8.61,27817,2
2,A013010002908283134592527,78643046,0,57.64,2017-02-01 07:47:38,1,0,0,0,549,0.000000,0.000000,0.000000,73.118280,52.076034,0,1,4.000000,4.0,4.0,57.64,0.00,28058,2
3,A011010002908105209831316,78643047,0,54.29,2017-02-01 07:48:48,0,1,1,1,267,0.000000,0.000000,0.000000,110.056926,53.554234,0,1,4.000000,4.0,4.0,54.29,0.00,28128,2
4,A013010041908000125652029,78643048,0,26.90,2017-02-01 08:13:27,1,0,0,0,549,0.003769,8.586333,0.001192,45.368313,52.076034,1,1,1.997106,4.0,4.0,59.15,32.25,29607,2


In [ ]:
to_discard = ['ZIBZIN', 'IDAvisAutorisationCheque', 'DateTransaction','CodeDecision']

# Discard unneeded columns
X = df.drop(columns=to_discard + ['Fraud'])
y = df['Fraud']

Exemple of pipeline
```python
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_predict
import numpy as np

# Base model 1: full dataset
model_full = RandomForestClassifier(random_state=42)

# Base model 2: undersampled
model_under = Pipeline([
    ('rus', RandomUnderSampler(random_state=42)),
    ('clf', GradientBoostingClassifier())
])

# Get OOF predictions
oof_full = cross_val_predict(model_full, X_train, y_train, cv=5, method='predict_proba')[:,1]
oof_under = cross_val_predict(model_under, X_train, y_train, cv=5, method='predict_proba')[:,1]

# Stack predictions as features for meta-model
X_meta = np.column_stack([oof_full, oof_under])

meta_model = LogisticRegression()
meta_model.fit(X_meta, y_train)

# Test predictions
pred_full_test = model_full.fit(X_train, y_train).predict_proba(X_test)[:,1]
pred_under_test = model_under.fit(X_train, y_train).predict_proba(X_test)[:,1]
X_meta_test = np.column_stack([pred_full_test, pred_under_test])
final_pred = meta_model.predict(X_meta_test)
```